In [ ]:
import numpy as np

Enunciado exercício class: <br>
Peça para o usuário uma lista com as comidas favoritas dele, depois transforme isso em um objeto numpy. Agora transformado, elimine todas as comidas que começam com uma vogal.

In [ ]:
comidas = np.array(['maçã', 'uva', 'morango', 'melancia', 'acelga'])
comidas

In [ ]:
[item[0] for item in comidas ]

In [ ]:
comidas[np.char.endswith(comidas, 'a')]

In [ ]:
[item for item in comidas if item[0] not in ['a','e','i','o','u']]

In [ ]:
prefixes = ["xyz", "abc"]
"abcde".startswith(tuple(prefixes))

In [ ]:
prefixes = ["a", "e", "i"]

[item for item in comidas if not item.startswith(tuple(prefixes))]

# Aula 3 - Limpeza e transformação de dados

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Limpeza de Dados (dropna, fillna, isnull, notnull, replace, duplicated, drop_duplicates)
- 2) Transformação de Dados (cut, qcut, get_dummies)

________

### Objetivos

Apresentar como identificamos valores nulos, discutir melhores formas de tratá-los além de trabalhar com variáveis dummies.

________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como identificar valores nulos e como tratá-los.
- Identificar quando utilizar variáveis dummies e como criá-las.
____
____
____

In [2]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv("data/titanic.csv")

## Limpeza de Dados: Dados Faltantes

### `.isna() e isnull()`
retorna um df booleano indicando se existe um nulo naquela posição. `pd.isnull()` é um alias para `pd.isna()` dentro do python como podemos ver na <a href="https://github.com/pandas-dev/pandas/blob/0409521665bd436a10aea7e06336066bf07ff057/pandas/core/dtypes/missing.py#L109">documentação</a>. O pandas dataframe é baseado nos df do R, onde null e na tem sentidos distintos.

In [ ]:
pd.isnull

Tanto o `.isna()` quanto o `.isnull()` retornam um df booleano:

In [ ]:
df.isna()

Para termos a quantidade de nans em cada coluna podemos somar os valores true:

In [ ]:
df.isna().sum()

### `.notnull()`

retorna df com True ou False, com True para elementos não nulos

In [ ]:
df.notnull().sum()

In [ ]:
df.notna().sum()

### `.dropna()`

elimina nulos em todo o df ou em colunas especificadas quando utilizado o argumento `subset`. É um dos métodos que aceita o parâmetro `inplace`.


In [ ]:
df.shape

In [ ]:
df.dropna().shape

In [ ]:
df.dropna(subset=['Age','Embarked']).shape

In [ ]:
df.dropna(subset=['Age','Embarked']).isna().sum()

In [ ]:
pd.DataFrame(df.dropna(subset=['Age','Embarked']).isna().sum(), columns= ['NaN_atualizado'])

### `.fillna()`

preeche os valores nulos com valores especificados.
Parâmetros úteis:

   * method {‘bfill’,‘ffill’, None}, default None <br>
       Preenche os NaNs propagando o último valor válido para frente (ffill) ou utiliza próxima observação válida para preencher os nulos (bfill).
   * axis {0 para ‘index’, 1 para ‘colunas’} <br>
       Eixo no qual se quer preencher os nulos
   * inplace {booleano}, default False


In [ ]:
fill = pd.DataFrame({'original': [np.nan,1,2,np.nan,4,5,np.nan,np.nan, np.nan, 9, 10, np.nan]})
fill

In [ ]:
fill.original.mean()

In [ ]:
fill['cte'] = fill.original.fillna(999)
fill['media'] = fill.original.fillna(fill.original.mean())
fill['ffill'] = fill.original.fillna(method='ffill')
fill['ffill_limit'] = fill.original.fillna(method='ffill', limit=2)
fill['bfill'] = fill.original.fillna(method='bfill')

fill

### Voltando ao dataset do titanic...
Quais soluções poderíamos utilizar para preencher os nulos em Age? <br>
E as cabines nulas? <br>
E os portões de embarque nulos? <br>
Qual os possíveis problemas que podemos encontrar utilizando essas abordagens? <br>


_________
_________
**Exercício:** Ordene as colunas Pclass e Fare, nessa ordem e ascendente, e preencha os nulos utilizando o método foward fill.

In [ ]:
df2 = df.sort_values(by = ['Pclass', 'Fare'], ascending = [True, True]).copy()
df2.fillna(method = 'ffill')

In [ ]:
df['new_cabin'] = df.sort_values(["Pclass", "Fare"])['Cabin'].fillna(method="ffill")

_________
_________

### `.interpolate()`
Faz a interpolação dos missing de acordo com o método escolhido.
[Documentação](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.interpolate.html)

In [ ]:
s = pd.DataFrame([0, 1, np.nan, 3, 4, np.nan, 400], columns=['original'])
s['linear'] = s.original.interpolate()
s['segundo_grau'] = s.original.interpolate(method="polynomial", order=2)
s

### `.replace()`

substitui elementos dentro do df. É um dos métodos que aceita o parâmetro `inplace`.


In [ ]:
# para apenas uma troca
df.replace({'h':'male'}, inplace=True)

In [ ]:
df.head(10)

In [ ]:
df.Sex.replace({'male':'H', 'female':'M'})

In [ ]:
df.head()

In [ ]:
# para mais de uma troca
df.replace({'Sex':{'male':'H', 'female':'M'}, 'Name':{'Braund, Mr. Owen Harris':'fera'}})

___________________
___________________

**Exercício**: Troque os valores abreviados do portão de embarque para seus respectivos nomes 
(C = Cherbourg; Q = Queenstown; S = Southampton) e preencha os dados faltantes com o portão mais frequente

In [93]:
df = pd.read_csv("data/titanic.csv")
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
df['Embarked'].replace({'C':'Cherbourg', 'Q':'Queenstown', 'S':'Southampton'}, inplace = True)
df.fillna(df['Embarked'].mode(), inplace = True)

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df.Embarked = df.replace({'Embarked': 
            {'C': 'Cherbourg', 'Q':'Queenstown', 'S':'Southampton'}
           })['Embarked'].fillna(df.Embarked.mode())

In [ ]:
df.head()

In [ ]:
df.isna().sum()

___________________
___________________


## Limpeza de Dados: Dados duplicados
### `.duplicated()` e `.drop_duplicated()`

O `.duplicated()` retorna uma series indicando se determinada linha possui duplicados ou não. Já o `.drop_duplicated()`, elimina as linhas duplicadas. <br>
Parâmetros do `.drop_duplicated()`:

   * `subset` -  seleciona colunas para serem utilizadas na comparação de linhas duplicadas
   * `keep` - {‘first’, ‘last’, False} (default ‘first’) Determina qual duplicado manter
   * `inplace` - (default False) Se False retorna uma cópia do df com as alterações, se True faz as modificações no próprio df



In [94]:
# vamos criar um df que possua linhas duplicadas
df_dup = df.copy()

df_dup.loc[891, :] = df_dup.loc[0, :]
df_dup.loc[892, :] = df_dup.loc[1, :]
df_dup.loc[893, :] = df_dup.loc[1, :]

print(df.shape, df_dup.shape)

(891, 12) (894, 12)


In [95]:
df_dup

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S
3,4.0,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5.0,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
889,890.0,1.0,1.0,"Behr, Mr. Karl Howell",male,26.0,0.0,0.0,111369,30.0000,C148,C
890,891.0,0.0,3.0,"Dooley, Mr. Patrick",male,32.0,0.0,0.0,370376,7.7500,NaN,Q
891,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
892,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C


In [98]:
df_dup.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
889    False
890    False
891     True
892     True
893     True
Length: 894, dtype: bool

In [ ]:
df_dup.drop_duplicates(subset=['Age', 'Embarked'])

In [ ]:
print(df_dup.shape)
print(df_dup.drop_duplicates().shape)

In [ ]:
%%timeit
df_dup.drop_duplicates(subset=['PassengerId'])

In [ ]:
%%timeit
df_dup.sort_values('PassengerId').drop_duplicates(subset=['PassengerId'])

_______________________
_______________________
**Exercícios:** Elimine os duplicados considerando as colunas 'brand' e 'style' mantendo o último valor.

In [ ]:
df_exerc = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, np.nan, 3.5, 15, 5]
})
df_exerc

In [ ]:
df_exerc.drop_duplicates(subset = ['brand', 'style'], keep = 'last')

In [ ]:
df_exerc2 = pd.DataFrame([
    ['Yum Yum', 'Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    ['cup', 'cup', 'cup', 'cup', 'pack', 'pack'],
    [4, 4, np.nan, 3.5, 15, 5]
])
df_exerc2.transpose()

In [ ]:
df_exerc

In [ ]:
df_exerc.drop_duplicates(subset=['brand','style'], inplace=True, keep='last')

In [ ]:
df_exerc

__________
__________


## Transformação de dados

### `.get_dummies()`

#### variáveis categóricas
Variáveis categóricas são aquelas que representam grupos ou classes dentro dos nossos dados. Elas podem ser de dois tipos:
* ordinais: possuem uma ordem que tem um sentido. Por exemplo, em rendimentos poderíamos ter: classe alta > classe média > classe baixa  
* nominais: não possuem uma ordem válida. Por exemplo: sexo e CEP.

<img src="variaveis_categoricas.jpeg" style="width: 500px">

Dummies são quaisquer variáveis cujos valores são 1 ou 0 para cada observação. O método `pd.get_dummies()` converte as variáveis categóricas em numéricas separando cada categoria em uma coluna única.
<br>
<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html">Parâmetros:</a> <br>
`pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)`

In [ ]:
pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

## Material de Aprofundamento

### `.cut()`
O método `.cut()` ordena os dados, separa em bins e computa qual grupo cada linha do df pertence. O `.cut()` escolherá os bins para serem espaçados uniformemente de acordo com os próprios valores e não com a frequência desses valores.  <br>
Ele é muito utilizado para transformar variáveis contínuas em categóricas. Por exemplo, podemos converter o valor númerico da idade em grupos de criança, jovem, adulto e idoso.
<br><br>
<a href='https://pandas.pydata.org/docs/reference/api/pandas.cut.html'>Parâmetros:</a> <br>
`pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)`

Ao informar quantidade de grupos o pd.cut() escolhe os bins com o mesmo tamanho de janela :


In [ ]:
df['cut_bins'] = pd.cut(df.Age, 4)
df.head()

In [ ]:
df.cut_bins.value_counts()

Podemos passar o nome dos grupos e transformar a variável numérica diretamente em categórica


In [ ]:
df['cut_classes'] = pd.cut(df.Age, 4, labels=["jovens", "adultos", "meia-idade", "idosos"])
df.head()

In [ ]:
df.cut_bins.unique()

In [ ]:
df.cut_classes.value_counts()

Também podemos passar uma lista com os valores de início e fim dos bins:

In [ ]:
pd.cut(df.Age, [0,20,60,80]).unique()

In [ ]:
df.Age.describe()

<a href='https://towardsdatascience.com/discretisation-using-decision-trees-21910483fa4b'>Discretização utilizando decision trees</a>

### `.qcut()`
O `pd.qcut()` é utilizado quando queremos discretizar nossos dados em quantis. Ao informar quantidade de grupos o `pd.qcut()` escolhe os bins tal que tenhamos a mesma quantidade de valores em cada grupo.

#### `.cut()` x `.qcut()`
   * O comando `.cut()` cria **caixas equidistantes**, mas a **frequência** das amostras é **desigual** em cada caixa
   * O comando `.qcut()` cria **caixas de tamanhos desiguais**, mas a **frequência** das amostras é **igual** em cada caixa.

<br>

[Parâmetros](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html)
<br>
`pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')`

In [ ]:
pd.cut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()/df.Age.notnull().sum()

## Exercícios

1. Converta os países do dataframe abaixo para variáveis dummies eliminando a primeira coluna e utilizando 'Country' como prefixo no nome das colunas.

In [ ]:
ids = [11, 22, 33, 44, 55, 66, 77]
countries = ['Brasil', 'Argentina', 'Peru', 'Bolívia', 'Uruguai']
sales = [42, 53, 37, 45, 20]

df = pd.DataFrame(list(zip(ids, countries, sales)),
                  columns=['Ids', 'Countries', 'Sales'])
df.head()

In [ ]:
pd.get_dummies(df, columns = ['Countries'], drop_first = True, prefix = 'Country')

2. Considere os dados do arquivo "german_credit.csv" que contem dados de empréstimos realizados por um banco.<br> <br>
a. Tem nulos nesse df? Se sim, preencha os nulos da forma que o grupo decidir.

In [85]:
df = pd.read_csv('./data/german_credit.csv', delimiter = ',')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.isna().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
dtype: int64

In [ ]:
df['Saving accounts'].value_counts()

In [ ]:
df['Checking account'].value_counts()

In [ ]:
df.head(20)

In [ ]:
df2 = df.copy()
df2['Cut_Credit amount'] = pd.qcut(df['Credit amount'],4, precision = 0)
df2[['Saving accounts', 'Cut_Credit amount']].drop_duplicates().set_index('Cut_Credit amount').sort_values(by = ['Cut_Credit amount', 'Saving accounts'], ascending = [True, True])
#nenhuma correlação aparente entre savings account e os quantis de credit amount

In [ ]:
df.isna().sum()

In [ ]:
df.size

In [ ]:
df.dropna().size # perde-se muitas linhas da base

In [ ]:
df.dropna(subset = ['Checking account']).size # ainda assim perde-se muitas linhas da base

In [ ]:
df.dropna(subset = ['Saving accounts']).size # ainda assim perde-se muitas linhas da base

In [64]:
df['Saving accounts'].value_counts(dropna = False)

little        603
NaN           183
moderate      103
quite rich     63
rich           48
Name: Saving accounts, dtype: int64

In [65]:
# Sendo assim o que faz mais sentido é substituir os NaNs pela moda de suas respectivas colunas
df['Saving accounts'].fillna(df['Saving accounts'].mode()[0], inplace = True, axis = 0)
df.isna().sum()


Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts       0
Checking account    394
Credit amount         0
Duration              0
Purpose               0
dtype: int64

In [66]:
df['Saving accounts'].value_counts(dropna = False)

little        786
moderate      103
quite rich     63
rich           48
Name: Saving accounts, dtype: int64

In [67]:
df['Checking account'].value_counts(dropna = False)

NaN         394
little      274
moderate    269
rich         63
Name: Checking account, dtype: int64

In [68]:
df['Checking account'].fillna(df['Checking account'].mode()[0], inplace = True)
df.isna().sum()

Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
dtype: int64

In [69]:
df['Checking account'].value_counts(dropna = False)

little      668
moderate    269
rich         63
Name: Checking account, dtype: int64

In [71]:
df.isna().sum()

Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
dtype: int64

In [ ]:
# Detalhes sobre a base german credit
# https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)

   b. Converta as variáveis categóricas em numéricas.

In [76]:
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,little,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,little,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


In [86]:
df_dum = pd.get_dummies(df, columns = ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose'], prefix = 'Dummy', drop_first = True)
df_dum

,Age,Job,Credit amount,Duration,Dummy_male,Dummy_own,Dummy_rent,Dummy_moderate,Dummy_quite rich,Dummy_rich,Dummy_moderate,Dummy_rich,Dummy_car,Dummy_domestic appliances,Dummy_education,Dummy_furniture/equipment,Dummy_radio/TV,Dummy_repairs,Dummy_vacation/others
0,67,2,1169,6,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,22,2,5951,48,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
2,49,1,2096,12,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,45,2,7882,42,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,53,2,4870,24,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,1,1736,12,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
996,40,3,3857,30,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
997,38,2,804,12,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
998,23,2,1845,45,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


3. Para esse exercício vamos considerar o dataset https://archive.ics.uci.edu/ml/datasets/automobile que traz a especificação de modelos de carros em 1985.
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

In [107]:
# Considere a seguinte lista como o nome das colunas do dataset
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

a. Esse dataset possui nans? Se sim, o grupo precisa entrar em um acordo de como tratá-los.

In [108]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data', delimiter = ',', names = headers)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [89]:
df.isna().sum() #Não possui

symboling            0
normalized_losses    0
make                 0
fuel_type            0
aspiration           0
num_doors            0
body_style           0
drive_wheels         0
engine_location      0
wheel_base           0
length               0
width                0
height               0
curb_weight          0
engine_type          0
num_cylinders        0
engine_size          0
fuel_system          0
bore                 0
stroke               0
compression_ratio    0
horsepower           0
peak_rpm             0
city_mpg             0
highway_mpg          0
price                0
dtype: int64

b. Esse dataset possui linhas duplicadas?

In [103]:
df.duplicated().sum() #Não possui

0

c. O consumo dos automóveis na cidade e rodovia, representados pelas colunas "city_mpg" e "highway_mpg" respectivamente, estão na unidade mpg (miles per gallon). Converta essas colunas para consumo em L/100km sabendo que a conversão é dada por:

1 L/100km = 235 mpg 

In [109]:
df['city_mpg'] = df['city_mpg']/235
df['highway_mpg'] = df['highway_mpg']/235
troca_nomes = {'city_mpg': 'city_L/100km', 'highway_mpg': 'highway_L/100km'}
df.rename(troca_nomes, axis = 1, inplace = True)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_L/100km,highway_L/100km,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,0.089362,0.114894,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,0.089362,0.114894,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,0.080851,0.110638,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,0.102128,0.127660,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,0.076596,0.093617,17450


d. Converta as colunas "body_style" e "drive_wheels" para variáveis dummies.

In [110]:
df = pd.get_dummies(df, columns = ['body_style', 'drive_wheels'], drop_first = True, prefix = 'Dummy')
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,engine_location,wheel_base,length,width,...,peak_rpm,city_L/100km,highway_L/100km,price,Dummy_hardtop,Dummy_hatchback,Dummy_sedan,Dummy_wagon,Dummy_fwd,Dummy_rwd
0,3,?,alfa-romero,gas,std,two,front,88.6,168.8,64.1,...,5000,0.089362,0.114894,13495,0,0,0,0,0,1
1,3,?,alfa-romero,gas,std,two,front,88.6,168.8,64.1,...,5000,0.089362,0.114894,16500,0,0,0,0,0,1
2,1,?,alfa-romero,gas,std,two,front,94.5,171.2,65.5,...,5000,0.080851,0.110638,16500,0,1,0,0,0,1
3,2,164,audi,gas,std,four,front,99.8,176.6,66.2,...,5500,0.102128,0.127660,13950,0,0,1,0,1,0
4,2,164,audi,gas,std,four,front,99.4,176.6,66.4,...,5500,0.076596,0.093617,17450,0,0,1,0,0,0


## Referências:
